<a href="https://colab.research.google.com/github/victorknox/rude-mood/blob/main/Rude_mood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [ ]:
!pip install psaw
!pip install praw
!pip install asyncpraw

In [128]:
import pandas as pd
pd.set_option('max_colwidth', 500)
pd.set_option('max_columns', 50)

In [129]:
from psaw import PushshiftAPI

api = PushshiftAPI()

In [130]:
import praw
reddit = praw.Reddit(client_id='zlMUUUA0MNikfTR-guoZPg', client_secret='vcgEzBynk1xZHyxaHdxDJ5v67M5_DQ', user_agent='scraper')

# Collection of data

In [131]:
def fetch_comments(url_list ):
  comments = []
  for x in url_list:
    submission = reddit.submission(url = x)
    submission.comments.replace_more(limit=0)
    for comment in submission.comments.list():
      comment_details = {
        'title':submission.title,
        'body':comment.body,
        'votes':comment.score,
        'subreddit name':submission.subreddit.display_name,
        # 'time':comment.created_utc,
        'author': comment.author,
        }
      comments.append(comment_details)
  return comments

## fetching windows data

In [132]:
api_request_generator = api.search_submissions(subreddit='windows', q='help', score = ">10")
win_submissions = pd.DataFrame([submission.d_ for submission in api_request_generator])
win_submissions.shape

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


(968, 99)

In [133]:
win_submissions.columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_css_class', 'author_flair_richtext', 'author_flair_text',
       'author_flair_type', 'author_fullname', 'author_patreon_flair',
       'author_premium', 'awarders', 'can_mod_post', 'contest_mode',
       'created_utc', 'domain', 'edited', 'full_link', 'gildings', 'id',
       'is_created_from_ads_ui', 'is_crosspostable', 'is_meta',
       'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable',
       'is_self', 'is_video', 'link_flair_background_color',
       'link_flair_css_class', 'link_flair_richtext', 'link_flair_template_id',
       'link_flair_text', 'link_flair_text_color', 'link_flair_type', 'locked',
       'media_metadata', 'media_only', 'no_follow', 'num_comments',
       'num_crossposts', 'over_18', 'parent_whitelist_status', 'permalink',
       'pinned', 'post_hint', 'preview', 'pwls', 'retrieved_on', 'score',
       'selftext', 'send_replies', 'spoiler', 'stickied', 'subreddit',

In [134]:
win_submissions[['title','selftext', 'num_comments', 'author','score','full_link','created_utc','subreddit']].sample(10)

,title,selftext,num_comments,author,score,full_link,created_utc,subreddit
774,"Tron v6.4.1 (2015-07-22) // RogueKiller (stalling, log size) fix; rkill whitelist fix; subtool updates","*[x-post /r/TronScript]*\n\n------------\n\n\n##Background\n\nTron is a script that ""fights for the User""; basically automates a bunch of scanning/disinfection/cleanup tools on a Windows system. I got tired of running these utilities manually and decided to just script the whole thing. I hope this helps other techs and admins.\n\n----------------------------------------\n\n##Stages of Tron:\n\n0. **Prep**: `rkill`, `ProcessKiller`, `TDSSKiller`, `Stinger`, `registry backup`, `WMI repair`, `s...",6,vocatus,23,https://www.reddit.com/r/windows/comments/3egaoz/tron_v641_20150722_roguekiller_stalling_log_size/,1437750543,windows
152,[Security] Someone tried to project on my computer?,"So i'm freaking out a bit because my PC has some rather important stuff and the last thing i need is to have my security compromised. I just got a windows notification that ""Mitu wants to project to your computer""... I clicked No but WTF?\n\nI am using windows 10 with Windows Defender on and Malwarebytes premium. I just scanned my computer again with both and got nothing.\n\nI mistakenly but publicly posted a screenshot with my IP a few hours ago and the only thing i can think of is someone ...",16,asdasdasdasdfff44,13,https://www.reddit.com/r/windows/comments/99hhhs/security_someone_tried_to_project_on_my_computer/,1534974421,windows
590,Friend commited suicide. family ask for help removing password from windows 10,"hi, before starting i want to say that this is 100% legitimate. My close friend committed suicide a week ago. his parents asked me if i could remove the password on his asus ROG gaming laptop for him. being that i've done this in the past with offline NT before, i said sure. well that was before windows 10. i tried using that and it seemed to work but still asks for a password, then i remember about windows 10 and having a hotmail account linked to it. (i use local) so its asking for the ho...",58,Valkyrie743,59,https://www.reddit.com/r/windows/comments/57933o/friend_commited_suicide_family_ask_for_help/,1476343417,windows
212,Literally locked out of my desktop. Please send help.,,32,AlexSilva247,27,https://www.reddit.com/r/windows/comments/8expgw/literally_locked_out_of_my_desktop_please_send/,1524695184,windows
226,System Locale Issues,So A couple months I wanted to play a Japanese game so I had to change the System Locale. I'm done with the game and so I switched the Locale back to English and now the Nvidia in my Control Panel under Appearance And Personalization is in some Japanese text and so is the Mcafee Security Scan Plus is now in Japanese too. Please help,3,mfright04,17,https://www.reddit.com/r/windows/comments/8avh14/system_locale_issues/,1523244431,windows
293,Trying to launch Steam with elevated privileges on boot/login,"But although I've followed several guides it doesn't seem to work. I'd appreciate any help from someone who has done this with a similar program.\n\nI login with a standard user but run programs with admin. This is why I would like this.\n\nAny help would be appreciated and if not, merry Christmas anywho.",15,ClandestineMovah,19,https://www.reddit.com/r/windows/comments/7m6suq/trying_to_launch_steam_with_elevated_privileges/,1514278882,windows
964,Should I finally make the switch from XP to Win7?,"A little background on myself. First, I tried using the search but didn't find a lot of advice specific to my situation, aside from most people supporting Win7 enough that I almost didn't feel the need to make this topic and instead just upgrade, so I apologize if this is old hat around here.\n\nSo, I have been using Mac computers for the last 4 years for school and work, so combining the fact that I have zero free time to play PC games, and that the last OS I was familiar with was XP Home, ...",56,Verb_Rogue,17,https://www.reddit

In [135]:
for x in win_submissions["full_link"][0:5]:
  print(x)

https://www.reddit.com/r/windows/comments/o0q4dz/if_you_want_to_run_the_windows_11_build_natively/
https://www.reddit.com/r/windows/comments/nzanlu/can_anyone_help_mei_just_came_back_from_linux_and/
https://www.reddit.com/r/windows/comments/lqxmtr/help_i_cant_tilt_it_out_and_lift_it_up_and_i/
https://www.reddit.com/r/windows/comments/lc9926/can_someone_help_me_to_please_solve_this_issue/
https://www.reddit.com/r/windows/comments/l9xqej/this_pops_up_when_im_trying_to_update_windows_i/


In [ ]:
win_comments = fetch_comments(win_submissions["full_link"][0:100]) 

In [137]:
import csv

keys = win_comments[0].keys()
with open('win_comments.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(win_comments)

## Fetching Linux data

In [138]:
api_request_generator = api.search_submissions(subreddit='linux', q='help', score = ">10")
lin_submissions = pd.DataFrame([submission.d_ for submission in api_request_generator])
lin_submissions.shape

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)


(1092, 96)

In [139]:
lin_submissions[['title','selftext', 'num_comments', 'author','score','full_link','created_utc','subreddit']].sample(10)

,title,selftext,num_comments,author,score,full_link,created_utc,subreddit
180,"With your help, we’re happy to introduce DigitalOcean Currents - A quarterly report on developer cloud trends","Hey Everyone! Last month we at DigitalOcean reached out and asked you all to participate in a survey asking about the tools and resources developers prefer. Over a couple weeks we received responses from over one thousand people. \n\nToday, we’re happy to announce the release of the first quarterly DigitalOcean Currents Report. Each quarter we’ll be sharing both survey results and a few pieces of information from our own internal systems that highlight how developers work. \n\n[The full ...",5,ryanq-do,13,https://www.reddit.com/r/linux/comments/71llgu/with_your_help_were_happy_to_introduce/,1506022327,linux
254,Torrent RSS with linux iso's,"Hi guy's. Does anyone got link to torrent RSS channel with only linux iso's ? I would like to help spreading ""the word"" by seeding some distros on my dedicated servers.",16,PonuryTyp,11,https://www.reddit.com/r/linux/comments/63zxe6/torrent_rss_with_linux_isos/,1491566270,linux
677,Is Linux right for me?,"I've been reading about how much safer, faster, and more stable this OS is but still am hesitant and have questions. Is it possible to have both Windows and Linux on the same computer? I've also heard of some people running the system off of a flash/USB drive...is that possible?\n\nHow easy would it be for someone to use this system that is code illiterate and doesnt have any experience with writing code?\n\nThanks a lot in advance for your time and help to anyone who helps.",87,wHispeRing-I,12,https://www.reddit.com/r/linux/comments/27p3fd/is_linux_right_for_me/,1402322545,linux
263,A preliminary Litebook review,"A few weeks back, you may recall that the folks from the Litebook project (https://litebook.store/) posted about their $250 Linux laptop running Elementary Linux. I ordered (along with the SSD for quicker boots) and it arrived today. I'm not much of a review writer, so I'll just put down my impressions and answer questions as I'm able.\n\nFirst impressions: The Litebook is a nice, compact little computer, on par in size and weight with a medium-sized Chromebook. I like having two USB 3.0...",9,retard_patrol,29,https://www.reddit.com/r/linux/comments/60yeoi/a_preliminary_litebook_review/,1490226620,linux
541,Gaming On Linux,"Since the Windows 10 went live, there have been various articles popping up all over Reddit, moaning about one problem or another with it. \n\nIn Redditor discussions, someone inevitably mentions Linux, and the thread generally turns into this:\n\n ""Linux""\n\n""But muh games""\n\n""Steam blah blah blah Wine""\n\n""what like 12 crappy non AAA games?""\n\nSo I would like to (with your help) make a list of games that run on Linux that I, or any one else, can point to when these things come up.\n\n**...",67,m0r05,38,https://www.reddit.com/r/linux/comments/3hcehx/gaming_on_linux/,1439836019,linux
77,Linux touchpad like a Macbook: progress and a call for help,,119,Austin98989,685,https://www.reddit.com/r/linux/comments/b5ekyj/linux_touchpad_like_a_macbook_progress_and_a_call/,1553539443,linux
114,Using Linux to teach PLCs,"I mostly use Allen Bradley PLCs, which only supports windows. I have an opportunity set up a new PLC lab for training new students PLCs. The class is not brand specific and I would love to use Open Source whenever possible. I will want to build ladder logic, download the programs onto the PLCs and operate inputs and outputs. If anyone has used Linux or knows software that might help me, let me know.",4,Finniss,16,https://www.reddit.com/r/linux/comments/8zzscn/using_linux_to_teach_plcs/,1531950594,linux
690,Mediagoblin free media streaming/hosting software needs your help.,,6,[deleted],72,https://www.reddit.com/r/linux/comments/1z24av/mediagoblin_free_media_streaminghosting_software/,1393475057,linux
605,Funny Linux Commands,"I haven't seen one of these thr

In [ ]:
lin_comments = fetch_comments(lin_submissions["full_link"][0:100]) 

In [141]:
import csv

keys = lin_comments[0].keys()
with open('lin_comments.csv', 'w', newline='')  as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(lin_comments)